Dara Corr - 22275193 - Etivity 4

In [ ]:
!pip install textblob
!pip install jellyfish
from textblob import TextBlob
import re

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
#task1
#Use the TextBlob Library (https://textblob.readthedocs.io/en/dev/) to write a function that takes a string as input and returns a list of (word, confidence) tuples with spelling suggestions.
from textblob import Word

def GetWordCorrections(word):
  typo = Word(word)
  corrections_list = typo.spellcheck()

  return corrections_list

GetWordCorrections("acress")


[('across', 0.6851851851851852),
 ('access', 0.1728395061728395),
 ('acres', 0.1111111111111111),
 ('actress', 0.021604938271604937),
 ('caress', 0.009259259259259259)]

In [ ]:
#task 2

#Use the TextBlob Library (https://textblob.readthedocs.io/en/dev/) to write a function that takes a sentence as input and corrects its spelling. 

def GetSentenceCorrected(sentence):
  sentencelist = sentence.split(' ')
  corrected_sentence = []
  for word in sentencelist:
    corrections_list = GetWordCorrections(word)
    chosen_correction = corrections_list[0][0]
    corrected_sentence.append(chosen_correction)

  outputsentence = " ".join(corrected_sentence)

  return outputsentence

print(GetSentenceCorrected("I have good speling"))

I have good spelling


In [ ]:
#task 3
import pandas as pd

!wget https://norvig.com/ngrams/count_1w.txt
filepath2 = "/content/count_1w.txt"
unigrams_df = pd.read_csv(filepath2, delimiter = '\t', header=None, keep_default_na=False) #"keep_default_na=False" added to code so for ex. "nulld" will be corrected to null 
unigrams_df.columns = ['unigram','counts']

unigrams_df.head(100)

--2022-10-30 16:23:12--  https://norvig.com/ngrams/count_1w.txt
Resolving norvig.com (norvig.com)... 158.106.138.13
Connecting to norvig.com (norvig.com)|158.106.138.13|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4956241 (4.7M) [text/plain]
Saving to: ‘count_1w.txt.2’

count_1w.txt.2      100%[===================>]   4.73M  5.05MB/s    in 0.9s    

2022-10-30 16:23:14 (5.05 MB/s) - ‘count_1w.txt.2’ saved [4956241/4956241]



,unigram,counts
0,the,23135851162
1,of,13151942776
2,and,12997637966
3,to,12136980858
4,a,9081174698
...,...,...
95,like,520585287
96,service,519537222
97,x,508609523
98,than,502609275


In [ ]:
#create a simple spelling correction function that takes a non-word string as input (i.e. "acress")
#and prints out a list of candidate corrections:

import jellyfish as jellyfish
import numpy as np

def nonWordSpellingCorrection(nonWord):
  #find all words which are within 1 distance of the nonWord using Damerau-Levenshtein Distance
  
  print("nonWord = \'{0}\'".format(nonWord))

  unigram_list = unigrams_df.unigram.values.tolist()
  count_list = unigrams_df.counts.values.tolist()
  totalunigramcounts = unigrams_df['counts'].sum()
 
  CorrectionCandidates = []
  CandidatesCounts = []

  for i in range(len(unigram_list)):
    if jellyfish.damerau_levenshtein_distance(nonWord,str(unigram_list[i])) == 1:
      CorrectionCandidates.append(unigram_list[i])
      CandidatesCounts.append(count_list[i])

  Prob_list = CandidatesCounts/totalunigramcounts
  
  pd.set_option('display.float_format', '{:.8f}'.format) #supress scientific notation for probability output

  #put data back into df
  output_df = pd.DataFrame(
      {'word': CorrectionCandidates,
     'frequency': CandidatesCounts,
     'P(word)': Prob_list
    })

  return output_df

nonWordSpellingCorrection("acress")

nonWord = 'acress'


,word,frequency,P(word)
0,access,217986984,0.00037065
1,across,76597151,0.00013024
2,acres,14208905,0.00002416
3,actress,7010056,0.00001192
4,adress,984657,0.00000167
5,caress,590047,0.00000100
6,cress,279364,0.00000048
7,apress,202431,0.00000034
8,acess,171785,0.00000029


In [ ]:
nonWordSpellingCorrection("nulld") # added "keep_default_na=False" when importing unigrams df so that "null" would be listed as a suggestion for "nulld" 


nonWord = 'nulld'


,word,frequency,P(word)
0,null,30739157,0.00005227
1,nulla,263140,0.00000045
2,nulls,182242,0.00000031
3,nulled,19203,0.00000003
4,nulle,15048,0.00000003
